# Prepare environment

In [ ]:
import numpy as np
import torch
import torchvision.transforms as T
import torch.nn.functional as F
from torch import nn
from torch.utils.data import DataLoader, Subset
from torchvision import datasets

# Stratified split
from sklearn.model_selection import train_test_split

# PyTorch TensorBoard support
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime

import matplotlib
import matplotlib.pyplot as plt
font = {'size' : 18}
matplotlib.rc('font', **font)

# Prepare datasets

In [ ]:
# Download training data from open datasets.
train_ds = datasets.CIFAR10(
    root="./data",
    train=True,
    download=True,
    transform=T.ToTensor(),
)

# Download test data from open datasets.
test_ds = datasets.CIFAR10(
    root="./data",
    train=False,
    download=True,
    transform=T.ToTensor(),
)

# Preparing for validaion test
train_indices, valid_indices, _, _ = train_test_split(
    range(len(train_ds)),
    train_ds.targets,
    test_size=10000
)
valid_ds = Subset(train_ds, valid_indices)
train_ds = Subset(train_ds, train_indices)

# Create data loaders
batch_size = 64
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
valid_dl = DataLoader(valid_ds, batch_size=batch_size, shuffle=False)
test_dl = DataLoader(test_ds, batch_size=batch_size, shuffle=False)

# Class mapping
classes = [
    'Airplane',
    'Automobile',
    'Bird',
    'Cat',
    'Deer',
    'Dog',
    'Frog',
    'Horse',
    'Ship',
    'Truck',
]

In [ ]:
# Report split sizes
print(f'Training set: {len(train_ds)} examples')
print(f'Validation set: {len(valid_ds)} examples')
print(f'Test set: {len(test_ds)} examples')

It is always a good idea to visualize the input and the label obtained from the DataLoader to make sure that the input data are as expected.

In [ ]:
# Get one batch of data
for batch in train_dl:
    imgs, lbls = batch
    break

print(imgs.shape)
print(lbls.shape)

In [ ]:
figure = plt.figure(figsize=(8, 8))
cols, rows = 3, 3
for i in range(1, cols * rows + 1):
    sample_idx = torch.randint(len(imgs), size=(1,)).item()
    img, label = imgs[sample_idx], lbls[sample_idx]
    figure.add_subplot(rows, cols, i)
    plt.title(classes[label])
    plt.axis("off")
    plt.imshow(img.numpy().transpose(1,2,0))
plt.show()

# Create a model

In [ ]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

# Define model
class SimpleCNN(nn.Module):
    def __init__(self):
        super().__init__()
        # YOUR CODE HERE

    def forward(self, x):
        # out = # YOUR CODE HERE
        return out

model = SimpleCNN().to(device)
print(model)

Once you have created a model, it is always a good idea to try to process the input data from the DataLoader to make sure that we can train the model.

In [ ]:
# Get one batch of data
for batch in train_dl:
    imgs, lbls = batch
    break

preds = model(imgs)
print(preds.shape)

# Train a model

In [ ]:
# Specify loss
# Note: Make sure to check the following when choosing the loss
#   1. the label format of each loss (one-hot, ordinal, etc.)
#   2. classification or regression
loss_fn = nn.CrossEntropyLoss()

In [ ]:
# Specify optimizer
# e.g., SGD, Adam, RMSProp, etc.
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
def train_one_epoch(dataloader, model, loss_fn, optimizer):
    # Size of the dataset
    size = len(dataloader.dataset)

    # Training mode
    model.train()

    # Keep track of the loss
    running_loss = 0.
    last_loss = 0.

    # We use enumerate to track the batch index
    for i, batch in enumerate(dataloader):
        X, y = batch
        X, y = X.to(device), y.to(device)

        # Zeros the optimizer's gradient
        optimizer.zero_grad()

        # Prediction
        pred = model(X)

        # Compute loss
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()  # calculate the backward gradients
        optimizer.step()  # adjust model's weights based on the observed gradients

        # Keep track of the loss
        running_loss += loss.item()
        if (i+1) % 50 == 0:
            last_loss = running_loss / 50
            current = (i + 1) * len(X)
            print(f"loss: {last_loss:>7f}  [{current:>5d}/{size:>5d}]")
            running_loss = 0.

In [ ]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)

    # Test mode
    model.eval()

    # Predict on test set
    loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            # Prediction
            pred = model(X)
            y_pred = pred.argmax(1)
            # Compute loss
            loss += loss_fn(pred, y).item()
            # Correct predictions
            correct += (y_pred == y).type(torch.float).sum().item()

    # Average loss
    loss /= num_batches

    # Accuracy
    accuracy = correct / size

    return loss, accuracy

The training process is conducted over several iterations (*epochs*). During each epoch, the model learns
parameters to make better predictions. We print the model's accuracy and loss at each epoch; we'd like to see the
accuracy increase and the loss decrease with every epoch.
We also keep track of the validation loss, and save the model that perform best on the validation set.



In [ ]:
epochs = 5
best_vloss = 100000.
hist = {
    'train_loss': [],
    'train_acc': [],
    'val_loss': [],
    'val_acc': [],
}

for epoch in range(epochs):
    print(f"Epoch {epoch+1}\n-------------------------------")
    train_one_epoch(train_dl, model, loss_fn, optimizer)
    train_loss, train_acc = test(train_dl, model, loss_fn)
    val_loss, val_acc = test(valid_dl, model, loss_fn)

    # History for learning curve
    hist['train_loss'].append(train_loss)
    hist['train_acc'].append(train_acc)
    hist['val_loss'].append(val_loss)
    hist['val_acc'].append(val_acc)

    # Track best performance, and save the model's state
    if val_loss < best_vloss:
        best_vloss = val_loss
        torch.save(model.state_dict(), 'model_best_vloss.pth')
        print('Saved best model to model_best_vloss.pth')
print("Done!")

In [ ]:
fig, ax = plt.subplots(figsize=(8,6))
ax.plot(hist['train_loss'], label='train')
ax.plot(hist['val_loss'], label='valid')
ax.set_ylabel('Loss')
ax.set_xlabel('Epochs')
plt.legend()
plt.show()

fig, ax = plt.subplots(figsize=(8,6))
ax.plot(hist['train_acc'], label='train')
ax.plot(hist['val_acc'], label='valid')
ax.set_ylabel('Accuracy')
ax.set_xlabel('Epochs')
plt.legend()
plt.show()

plt.close('all')

# Load Best Model on Validation Set



In [ ]:
model_best = SimpleCNN().to(device)
model_best.load_state_dict(torch.load("model_best_vloss.pth"))

# Evaluate on the test set

Predict on one example.



In [ ]:
model_best.eval()

# Get one batch of data
for batch in test_dl:
    imgs, lbls = batch
    break

X, y = imgs[0:1].to(device), lbls[0:1].to(device)
with torch.no_grad():
    pred = model_best(X)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predict on the whole test set.

In [ ]:
# Use the best model on the validation set
test_loss, test_acc = test(test_dl, model_best, loss_fn)
print(f"Test: loss={test_loss:>8f}, acc={(100*test_acc):>0.1f}%")

In [ ]:
# Use the latest model
# Note: the latest model might be different from the best on validation set
test_loss, test_acc = test(test_dl, model, loss_fn)
print(f"Test: loss={test_loss:>8f}, acc={(100*test_acc):>0.1f}%")